In [86]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")

import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
pyoff.init_notebook_mode(connected=True)

In [63]:

df = pd.read_csv(r"C:\Users\muhad\DAV\formerge\Call_duration\Call_Combined.csv")
df.head()

,timestamp,CALLS_date,CALLS_duration,User
0,3/23/2013 22:24,3/23/2013,10,1
1,3/23/2013 22:24,3/23/2013,13,1
2,3/23/2013 22:24,3/23/2013,18,1
3,3/23/2013 22:24,3/23/2013,438,1
4,3/23/2013 22:24,3/22/2013,22,1


In [64]:
from datetime import datetime
from dateutil.parser import parse
import pandas as pd
#datetime.datetime.strptime(df[['CALLS_date']], '%m/%d/%Y')
#df['CALLS_date']
new=pd.to_datetime(df['CALLS_date'])

In [65]:
df= df.drop(['timestamp', 'User'], axis=1)

In [69]:
df

,CALLS_date,CALLS_duration
0,3/23/2013,10
1,3/23/2013,13
2,3/23/2013,18
3,3/23/2013,438
4,3/22/2013,22
...,...,...
6382,6/6/2013,0
6383,6/6/2013,30
6384,6/6/2013,241
6385,6/6/2013,13


In [70]:
df['Date'] = df['CALLS_duration'].map(new)
print (df)

     CALLS_date  CALLS_duration       Date
0     3/23/2013              10 2013-03-20
1     3/23/2013              13 2013-03-17
2     3/23/2013              18 2013-03-15
3     3/23/2013             438 2013-05-27
4     3/22/2013              22 2013-03-14
...         ...             ...        ...
6382   6/6/2013               0 2013-03-23
6383   6/6/2013              30 2013-03-13
6384   6/6/2013             241 2013-04-12
6385   6/6/2013              13 2013-03-17
6386   6/7/2013               0 2013-03-23

[6387 rows x 3 columns]


In [71]:
df

,CALLS_date,CALLS_duration,Date
0,3/23/2013,10,2013-03-20
1,3/23/2013,13,2013-03-17
2,3/23/2013,18,2013-03-15
3,3/23/2013,438,2013-05-27
4,3/22/2013,22,2013-03-14
...,...,...,...
6382,6/6/2013,0,2013-03-23
6383,6/6/2013,30,2013-03-13
6384,6/6/2013,241,2013-04-12
6385,6/6/2013,13,2013-03-17


In [72]:
tab = pd.read_csv(r'C:\Users\muhad\DAV\Level_2\Book1.csv')
tab 

,CALLS_date,CALLS_duration
0,3/23/2013,10
1,3/23/2013,13
2,3/23/2013,18
3,3/23/2013,438
4,3/22/2013,22
...,...,...
518,6/14/2013,8
519,6/14/2013,6
520,6/14/2013,0
521,6/14/2013,0


In [73]:
new=pd.to_datetime(tab['CALLS_date'])

In [74]:
new

0     2013-03-23
1     2013-03-23
2     2013-03-23
3     2013-03-23
4     2013-03-22
         ...    
518   2013-06-14
519   2013-06-14
520   2013-06-14
521   2013-06-14
522   2013-06-14
Name: CALLS_date, Length: 523, dtype: datetime64[ns]

In [76]:
tab['Date'] = new

In [79]:
tab=tab.drop(['CALLS_date'], axis=1)

In [80]:
tab.

,CALLS_duration,Date
0,10,2013-03-23
1,13,2013-03-23
2,18,2013-03-23
3,438,2013-03-23
4,22,2013-03-22
...,...,...
518,8,2013-06-14
519,6,2013-06-14
520,0,2013-06-14
521,0,2013-06-14


In [84]:
tab.set_index('Date')

,CALLS_duration
Date,
2013-03-23,10
2013-03-23,13
2013-03-23,18
2013-03-23,438
2013-03-22,22
...,...
2013-06-14,8
2013-06-14,6
2013-06-14,0


In [111]:
def parser(x):
    return pd.datetime.strptime(''+x,'%m/%d/%Y')

df = pd.read_csv(r'C:\Users\muhad\DAV\Level_2\Book1.csv', parse_dates=[0], index_col=0, date_parser=parser)
df.tail()

,CALLS_duration
CALLS_date,
2013-06-14,8
2013-06-14,6
2013-06-14,0
2013-06-14,0
2013-06-14,0


In [112]:
train = df

scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)

In [113]:
n_input = 12
n_features = 1
generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)

In [114]:
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
model.add(Dropout(0.15))
model.add(Dense(1))

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')

In [115]:
history = model.fit_generator(generator,epochs=100,verbose=1)

Epoch 1/100
86/86 [==============================] - 3s 7ms/step - loss: 0.0067
Epoch 2/100
86/86 [==============================] - 1s 7ms/step - loss: 0.0065
Epoch 3/100
86/86 [==============================] - 1s 7ms/step - loss: 0.0065
Epoch 4/100
86/86 [==============================] - 1s 7ms/step - loss: 0.0065
Epoch 5/100
86/86 [==============================] - 1s 7ms/step - loss: 0.0065
Epoch 6/100
86/86 [==============================] - 1s 7ms/step - loss: 0.0065
Epoch 7/100
86/86 [==============================] - 1s 7ms/step - loss: 0.0065
Epoch 8/100
86/86 [==============================] - 1s 7ms/step - loss: 0.0065
Epoch 9/100
86/86 [==============================] - 1s 7ms/step - loss: 0.0064
Epoch 10/100
86/86 [==============================] - 1s 7ms/step - loss: 0.0065
Epoch 11/100
86/86 [==============================] - 1s 11ms/step - loss: 0.0064
Epoch 12/100
86/86 [==============================] - 1s 11ms/step - loss: 0.0065
Epoch 13/100
86/86 [===============

In [116]:

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch

plot_data = [
    go.Scatter(
        x=hist['epoch'],
        y=hist['loss'],
        name='loss'
    )
    
]

plot_layout = go.Layout(
        title='Training loss'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [117]:

pred_list = []

batch = train[-n_input:].reshape((1, n_input, n_features))

for i in range(n_input):   
    pred_list.append(model.predict(batch)[0]) 
    batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)

In [118]:
add_dates = [df.index[-1] + DateOffset(months=x) for x in range(0,13) ]
future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)

In [128]:
df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                          index=future_dates[-n_input:].index, columns=['Prediction'])

df_proj = pd.concat([df,df_predict])

df_proj.tail(12)

,CALLS_duration,Prediction
2013-07-14,NaN,80.539969
2013-08-14,NaN,87.791118
2013-09-14,NaN,89.790919
2013-10-14,NaN,90.311491
2013-11-14,NaN,89.597438
2013-12-14,NaN,88.838445
2014-01-14,NaN,88.287711
2014-02-14,NaN,87.975442
2014-03-14,NaN,87.841669
2014-04-14,NaN,87.821154


In [130]:
plot_data = [
    go.Scatter(
        x=df_proj.index,
        y=df_proj['CALLS_duration'],
        name='actual'
    ),
    go.Scatter(
        x=df_proj.index,
        y=df_proj['Prediction'],
        name='prediction'
    )
]

plot_layout = go.Layout(
        title='Shampoo sales prediction'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)